# Yijing Oracle Development Notebook

Dieses Notebook dient der systematischen Entwicklung und dem Testing des Yijing Oracle Moduls. Wir werden Schritt für Schritt die Komponenten testen und verfeinern.

## Inhaltsverzeichnis
1. Setup und Grundkonfiguration
2. Basismodelle testen (Linien, Hexagramme)
3. Orakel-Funktionalität testen
4. Integration mit GenAI API
5. End-to-End Tests

Legen wir los!

## 1. Setup und Grundkonfiguration

Zuerst importieren wir die notwendigen Module und setzen die Grundkonfiguration.

In [1]:
import os
import sys
from pathlib import Path
import logging

# Füge den Projektordner zum Python-Path hinzu
project_dir = Path.cwd().parent  # Passe dies an deine Struktur an
sys.path.append(str(project_dir))

# Konfiguriere Logging
logging.basicConfig(level=logging.DEBUG)

# Importiere das yijing Modul
from yijing.models import HypergramLine, Hexagram, Hypergram
from yijing.hypergram import cast_hypergram
from yijing.oracle import YijingOracle
from yijing.enums import ConsultationMode

print("Setup erfolgreich abgeschlossen!")

Setup erfolgreich abgeschlossen!


## 2. Basismodelle testen

Hier testen wir die grundlegenden Modelle wie HypergramLine, Hexagram etc.

In [2]:
# Test: HypergramLine erstellen und Eigenschaften prüfen
def test_hypergram_line():
    # Teste verschiedene Linientypen
    lines = [
        HypergramLine(value=6),  # Changing Yin
        HypergramLine(value=7),  # Stable Yang
        HypergramLine(value=8),  # Stable Yin
        HypergramLine(value=9)   # Changing Yang
    ]
    
    for line in lines:
        print(f"\nLine value: {line.value}")
        print(f"Is Yin: {line.is_yin()}")
        print(f"Is Yang: {line.is_yang()}")
        print(f"Is Changing: {line.is_changing()}")
        print(f"Unicode Symbol: {line.to_unicode_symbol()}")

test_hypergram_line()

DEBUG:yijing.models:Checking if line 6 is yin
DEBUG:yijing.models:Checking if line 6 is yang
DEBUG:yijing.models:Checking if line 6 is changing
DEBUG:yijing.models:Checking if line 6 is yang
DEBUG:yijing.models:Checking if line 7 is yin
DEBUG:yijing.models:Checking if line 7 is yang
DEBUG:yijing.models:Checking if line 7 is changing
DEBUG:yijing.models:Checking if line 7 is yang
DEBUG:yijing.models:Checking if line 8 is yin
DEBUG:yijing.models:Checking if line 8 is yang
DEBUG:yijing.models:Checking if line 8 is changing
DEBUG:yijing.models:Checking if line 8 is yang
DEBUG:yijing.models:Checking if line 9 is yin
DEBUG:yijing.models:Checking if line 9 is yang
DEBUG:yijing.models:Checking if line 9 is changing
DEBUG:yijing.models:Checking if line 9 is yang



Line value: 6
Is Yin: True
Is Yang: False
Is Changing: True
Unicode Symbol: ䷁

Line value: 7
Is Yin: False
Is Yang: True
Is Changing: False
Unicode Symbol: ䷀

Line value: 8
Is Yin: True
Is Yang: False
Is Changing: False
Unicode Symbol: ䷁

Line value: 9
Is Yin: False
Is Yang: True
Is Changing: True
Unicode Symbol: ䷀


## 3. Orakel-Funktionalität testen

Jetzt testen wir die Hauptfunktionalität des Orakels.

In [2]:
# Neue Zelle im Workbook

import os
import json
from pathlib import Path
from typing import Dict, Any, Optional
from functools import lru_cache

# Projektverzeichnis bestimmen
project_dir = Path.cwd()#.parent  # Ein Verzeichnis höher, da wir im workbook/ sind
resources_dir = project_dir / 'yijing' / 'resources'

@lru_cache(maxsize=64)  # Caching für bessere Performance
def load_hexagram_data(hexagram_number: int) -> Dict[str, Any]:
    """
    Lädt die JSON-Daten für ein spezifisches Hexagramm.
    
    Args:
        hexagram_number: Nummer des Hexagramms (1-64)
        
    Returns:
        Dict mit den Hexagramm-Daten
        
    Raises:
        FileNotFoundError: Wenn die Hexagramm-Datei nicht gefunden wird
    """
    hexagram_file = resources_dir / 'hexagram_json' / f'hexagram_{hexagram_number:02d}.json'
    
    if not hexagram_file.exists():
        raise FileNotFoundError(f"Hexagramm-Datei nicht gefunden: {hexagram_file}")
        
    with open(hexagram_file, 'r', encoding='utf-8') as f:
        return json.load(f)

# Test der Funktion und Verzeichnisstruktur
print(f"Projektverzeichnis: {project_dir}")
print(f"Resources Verzeichnis: {resources_dir}")
print(f"Resources Verzeichnis existiert: {resources_dir.exists()}")

try:
    hexagram_1 = load_hexagram_data(1)
    print(f"\nHexagramm 1 geladen: {hexagram_1['hexagram']['name']}")
except Exception as e:
    print(f"\nFehler beim Laden: {e}")

Projektverzeichnis: /Users/johanneskaindl/Documents/0_inbox/2_projekte/1_coding/YijingOracle
Resources Verzeichnis: /Users/johanneskaindl/Documents/0_inbox/2_projekte/1_coding/YijingOracle/yijing/resources
Resources Verzeichnis existiert: True

Hexagramm 1 geladen: GUAI / DER DURCHBRUCH


In [16]:
# New cell in workbook

from dataclasses import dataclass
from typing import Dict, Any, List, Optional, Tuple
from functools import lru_cache

@dataclass
class HexagramContext:
    """Represents the complete context for a hexagram reading."""
    original_hexagram: Dict[str, Any]
    changing_lines: List[int]
    resulting_hexagram: Dict[str, Any]
    
    def get_relevant_line_interpretations(self) -> List[Dict[str, str]]:
        """Returns interpretations for the changing lines."""
        relevant_lines = []
        for line_num in self.changing_lines:
            # Korrigiere den Index (line_num - 1, da die JSON-Liste bei 0 beginnt)
            array_index = line_num - 1
            if 0 <= array_index < len(self.original_hexagram['lines']):
                line_data = self.original_hexagram['lines'][array_index]
                relevant_lines.append({
                    'position': line_data['position'],
                    'text': line_data['text'],
                    'interpretation': line_data['interpretation']
                })
        return relevant_lines

class HexagramManager:
    """Manages loading and organizing hexagram data for readings."""
    
    def __init__(self, resources_path: Path):
        self.resources_path = resources_path / 'hexagram_json'
        if not self.resources_path.exists():
            raise FileNotFoundError(f"Hexagram resources directory not found: {self.resources_path}")
    
    @lru_cache(maxsize=64)
    def load_hexagram(self, hexagram_number: int) -> Dict[str, Any]:
        """Loads a specific hexagram's data with caching."""
        hexagram_file = self.resources_path / f'hexagram_{hexagram_number:02d}.json'
        
        if not hexagram_file.exists():
            raise FileNotFoundError(f"Hexagram file not found: {hexagram_file}")
            
        with open(hexagram_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    
    def create_reading_context(self, 
                             original_hex_num: int, 
                             changing_lines: List[int], 
                             resulting_hex_num: int) -> HexagramContext:
        """Creates a complete context for a hexagram reading."""
        return HexagramContext(
            original_hexagram=self.load_hexagram(original_hex_num),
            changing_lines=changing_lines,
            resulting_hexagram=self.load_hexagram(resulting_hex_num)
        )

    def get_consultation_prompt(self, context: HexagramContext, question: str) -> str:
        """
        Erstellt einen detaillierten Prompt für die KI-Beratung mit allen relevanten
        Aspekten der Hexagramme und deren Interpretationen.
        """
        # Korrekte Extraktion der Hexagramm-Informationen
        original_hex = context.original_hexagram
        resulting_hex = context.resulting_hexagram
        
        # Wandelnde Linien Text mit korrektem JSON-Pfad
        changing_lines_text = "\n".join([
            f"Linie {line['position']}: {line['text']}\n"
            f"Interpretation: {line['interpretation']}\n"
            for line in context.get_relevant_line_interpretations()
        ])
        
        return f"""Beratungsanfrage: {question}

AUSGANGSSITUATION - {original_hex['hexagram']['name']}
================================================================================
Grundbedeutung:
{original_hex['hexagram']['meaning']['description']}

Das Urteil:
{original_hex['judgment']['description']}

Das Bild:
{original_hex['image']['description']}

WANDELNDE LINIEN
================================================================================
{changing_lines_text}

ZUKÜNFTIGE TENDENZ - {resulting_hex['hexagram']['name']}
================================================================================
Grundbedeutung:
{resulting_hex['hexagram']['meaning']['description']}

Das Urteil:
{resulting_hex['judgment']['description']}

Das Bild:
{resulting_hex['image']['description']}

================================================================================
ANWEISUNG FÜR DIE BERATUNG:
Als weises I Ging Orakel wird um eine tiefgründige Interpretation und Beratung gebeten.
Bitte berücksichtige in deiner Antwort:

1. Die gegenwärtige Situation, wie sie sich im Ausgangshexagramm zeigt
2. Die bedeutsamen Wandlungen, die durch die sich verändernden Linien angezeigt werden
3. Die sich entwickelnde Tendenz, wie sie im resultierenden Hexagramm erscheint
4. Konkrete und praktische Handlungsempfehlungen für die ratsuchende Person

Formuliere die Antwort in einem weisen, aber verständlichen Ton. Beziehe dich auf die 
traditionellen Bilder und Symbole des I Ging, aber übersetze ihre Bedeutung in den 
modernen Kontext der fragenden Person.
"""

    # Test des verbesserten Prompts
    try:
        manager = HexagramManager(resources_dir)
        context = manager.create_reading_context(
            original_hex_num=1,
            changing_lines=[3, 6],
            resulting_hex_num=2
        )
        
        test_prompt = manager.get_consultation_prompt(
            context=context,
            question="Was sollte ich in meiner aktuellen Situation beachten?"
        )
        
        print(test_prompt)
        
    except Exception as e:
        print(f"Fehler beim Testen: {e}")

Beratungsanfrage: Was sollte ich in meiner aktuellen Situation beachten?

AUSGANGSSITUATION - GUAI / DER DURCHBRUCH
Grundbedeutung:
Das Zeichen bedeutet einerseits einen Durchbruch nach lange angesammelter Spannung, wie den Durchbruch eines geschwellten Flusses durch seine Dämme, wie einen Wolkenbruch. Auf menschliche Verhältnisse übertragen, ist es andererseits die Zeit, da allmählich die Gemeinen im Schwinden sind. Ihr Einfluß ist im Abnehmen, und durch eine entschlossene Aktion kommt eine Änderung der Verhältnisse zum Durchbruch.

Das Urteil:
Der Durchbruch. Entschlossen muß man am Hof des Königs die Sache bekanntmachen. Der Wahrheit gemäß muß sie verkündet werden. Gefahr ! Man muß seine eigene Stadt benachrichtigen. Nicht fördernd ist es, zu den Waffen zu greifen. Fördernd ist es, etwas zu unternehmen.

Das Bild:
Der See ist an den Himmel emporgestiegen: das Bild des Durchbruchs. So spendet der Edle Reichtum nach oben und teilt ihn nach unten.

WANDELNDE LINIEN
Linie Neun auf dritt

In [20]:
# import display and markdown
from IPython.display import display, Markdown

manager = HexagramManager(resources_dir)
context = manager.create_reading_context(
    original_hex_num=1,
    changing_lines=[3, 6],
    resulting_hex_num=2
)

test_prompt = manager.get_consultation_prompt(
    context=context,
    question="Was sollte ich in meiner aktuellen Situation beachten?"
)

display(Markdown(test_prompt))
test_prompt

Beratungsanfrage: Was sollte ich in meiner aktuellen Situation beachten?

AUSGANGSSITUATION - GUAI / DER DURCHBRUCH
================================================================================
Grundbedeutung:
Das Zeichen bedeutet einerseits einen Durchbruch nach lange angesammelter Spannung, wie den Durchbruch eines geschwellten Flusses durch seine Dämme, wie einen Wolkenbruch. Auf menschliche Verhältnisse übertragen, ist es andererseits die Zeit, da allmählich die Gemeinen im Schwinden sind. Ihr Einfluß ist im Abnehmen, und durch eine entschlossene Aktion kommt eine Änderung der Verhältnisse zum Durchbruch.

Das Urteil:
Der Durchbruch. Entschlossen muß man am Hof des Königs die Sache bekanntmachen. Der Wahrheit gemäß muß sie verkündet werden. Gefahr ! Man muß seine eigene Stadt benachrichtigen. Nicht fördernd ist es, zu den Waffen zu greifen. Fördernd ist es, etwas zu unternehmen.

Das Bild:
Der See ist an den Himmel emporgestiegen: das Bild des Durchbruchs. So spendet der Edle Reichtum nach oben und teilt ihn nach unten.

WANDELNDE LINIEN
================================================================================
Linie Neun auf drittem Platz: Mächtig an den Backenknochen bringt es Unheil. Der Edle ist entschlossen zum Durchbruch. Er wandelt allein und trifft auf Regen. Er wird befleckt, aber man wird ihm nicht zürnen.
Interpretation: Entschlossenheit kann in Aggressivität umschlagen und Unheil bringen. Der Edle bleibt auch in schwierigen Zeiten standhaft und wird trotz kleiner Fehler nicht getadelt.

Linie Oben eine Sechs: Kein Schreien. Unheil!
Interpretation: Der Durchbruch ist vollzogen, aber es fehlt an Besonnenheit und Ruhe. Übereiltes Handeln führt zu Unheil.


ZUKÜNFTIGE TENDENZ - SUN / DAS SANFTE
================================================================================
Grundbedeutung:
Sun ist eines der acht Doppelzeichen. Es ist die älteste Tochter, hat als Bild den Wind oder das Holz, als Eigenschaft die Sanftheit, die jedoch eindringt wie der Wind oder das Holz mit seinen Wurzeln. Das Dunkle, das an sich starr und unbeweglich ist, wird aufgelöst durch das eindringende lichte Prinzip, dem es sich unterordnet in Sanftheit. In der Natur ist es der Wind, der die angehäuften Wolken auseinandertreibt und heitre Himmelklarheit schafft. Im Menschenleben ist es die durchdringende Klarheit des Urteils, die alle dunklen Hintergedanken zunichte macht. Im Leben der Gemeinschaft ist es der mächtige Einfluß einer bedeutenden Persönlichkeit, die alle lichtscheuen Machenschaften aufdeckt und auseinandertreibt.

Das Urteil:
Das Sanfte. Durch Kleines Gelingen. Fördernd ist es, zu haben, wohin man geht. Fördernd ist es, den großen Mann zu sehen.

Das Bild:
Einander folgende Winde: das Bild des Sanft-Eindringenden. So verbreitet der Edle seine Gebote und wirkt seine Geschäfte.

================================================================================
ANWEISUNG FÜR DIE BERATUNG:
Als weises I Ging Orakel wird um eine tiefgründige Interpretation und Beratung gebeten.
Bitte berücksichtige in deiner Antwort:

1. Die gegenwärtige Situation, wie sie sich im Ausgangshexagramm zeigt
2. Die bedeutsamen Wandlungen, die durch die sich verändernden Linien angezeigt werden
3. Die sich entwickelnde Tendenz, wie sie im resultierenden Hexagramm erscheint
4. Konkrete und praktische Handlungsempfehlungen für die ratsuchende Person

Formuliere die Antwort in einem weisen, aber verständlichen Ton. Beziehe dich auf die 
traditionellen Bilder und Symbole des I Ging, aber übersetze ihre Bedeutung in den 
modernen Kontext der fragenden Person.


'Beratungsanfrage: Was sollte ich in meiner aktuellen Situation beachten?\n\nAUSGANGSSITUATION - GUAI / DER DURCHBRUCH\n================================================================================\nGrundbedeutung:\nDas Zeichen bedeutet einerseits einen Durchbruch nach lange angesammelter Spannung, wie den Durchbruch eines geschwellten Flusses durch seine Dämme, wie einen Wolkenbruch. Auf menschliche Verhältnisse übertragen, ist es andererseits die Zeit, da allmählich die Gemeinen im Schwinden sind. Ihr Einfluß ist im Abnehmen, und durch eine entschlossene Aktion kommt eine Änderung der Verhältnisse zum Durchbruch.\n\nDas Urteil:\nDer Durchbruch. Entschlossen muß man am Hof des Königs die Sache bekanntmachen. Der Wahrheit gemäß muß sie verkündet werden. Gefahr ! Man muß seine eigene Stadt benachrichtigen. Nicht fördernd ist es, zu den Waffen zu greifen. Fördernd ist es, etwas zu unternehmen.\n\nDas Bild:\nDer See ist an den Himmel emporgestiegen: das Bild des Durchbruchs. So spendet

In [ ]:
# API Key setzen (ersetze dies durch deinen tatsächlichen Key)
api_key = os.environ["API_KEY"]

# Oracle initialisieren
oracle = YijingOracle()

# Test: Einfache Anfrage
response = oracle.get_response("Was sollte ich heute beachten?")
print("Oracle Antwort:")
print(response['answer'])

FileNotFoundError: Yijing text file not found at /Users/johanneskaindl/Documents/0_inbox/2_projekte/1_coding/YijingOracle/yijing/resources/yijing.txt. Please ensure the file exists in the resources directory.

## 4. Integration mit GenAI API

Hier testen wir die Integration mit der Google GenAI API.

In [ ]:
# Test: Dialog-Modus
oracle_dialogue = YijingOracle(
    custom_settings={
        "consultation_mode": ConsultationMode.DIALOGUE
    }
)

# Starte eine neue Beratung
oracle_dialogue.start_new_consultation()
response = oracle_dialogue.get_response("Wie kann ich meine aktuelle Situation verbessern?")
print("Dialog-Antwort:")
print(response['answer'])

## 5. End-to-End Tests

Abschließend führen wir einige End-to-End Tests durch.

In [ ]:
def run_end_to_end_test():
    print("Starting End-to-End Test...\n")
    
    # 1. Hypergram werfen
    hypergram_data = cast_hypergram()
    print("Geworfenes Hypergram:")
    print(f"Altes Hexagramm: {hypergram_data.old_hexagram.to_unicode_representation()}")
    print(f"Neues Hexagramm: {hypergram_data.new_hexagram.to_unicode_representation()}")
    print(f"Wandelnde Linien: {hypergram_data.changing_lines}\n")
    
    # 2. Oracle Antwort generieren
    response = oracle.get_response("Wie soll ich mit Veränderungen umgehen?")
    print("Oracle Antwort:")
    print(response['answer'])

run_end_to_end_test()

## Nächste Schritte

1. Fehlerbehandlung verbessern
2. Tests für Edge Cases hinzufügen
3. Dokumentation erweitern
4. Performance optimieren

Notiere hier Beobachtungen und nötige Anpassungen: